# Initialize Morpheus

In [1]:
from morpheus.project.application.write import project_command_bus

# Create a Project

In [2]:
from morpheus.common.types.identity.Identity import UserId
from morpheus.project.types.Project import ProjectId, Name, Description, Tags

user_id = UserId.new()
project_id = ProjectId.new()
name = Name.from_str('Project 1')
description = Description.from_str('Description 1')
tags = Tags.from_list(['tag1', 'tag2'])

from morpheus.project.application.write.Project import CreateProjectCommand

create_project_command = CreateProjectCommand(
  project_id=project_id,
  name=name,
  description=description,
  tags=tags,
  user_id=user_id
)

project_command_bus.dispatch(create_project_command)

# Read ProjectList

In [3]:
from morpheus.common.types.identity.Identity import Identity
from morpheus.project.application.read.ProjectReader import project_reader

identity = Identity(user_id=UserId.new(), group_ids=[], is_admin=True)
project_list = project_reader.get_project_summaries_with_user_privileges_for_identity(identity)
project_list

[(ProjectSummary(project_id=ProjectId(value='256fbece-6fcf-43cc-9b52-363172dcabae'), project_name=Name(value='Notebook Project 1_0'), project_description=Description(value='This is project 1_0 created in a notebook'), project_tags=Tags(value=[]), owner_id=UserId(value='5b5a1041-a0df-4a27-8599-70aed87f8a5d'), visibility=<Visibility.PRIVATE: 'private'>, created_at=DateTime(value=datetime.datetime(2024, 6, 20, 9, 58, 16, 343155)), updated_at=DateTime(value=datetime.datetime(2024, 6, 20, 9, 58, 16, 343155))),
  [<Privilege.FULL_ACCESS: 'Full Access'>]),
 (ProjectSummary(project_id=ProjectId(value='c7efd6d9-e265-4dcc-915d-8625552d1b30'), project_name=Name(value='Notebook Project 1_1'), project_description=Description(value='This is project 1_1 created in a notebook'), project_tags=Tags(value=[]), owner_id=UserId(value='5b5a1041-a0df-4a27-8599-70aed87f8a5d'), visibility=<Visibility.PRIVATE: 'private'>, created_at=DateTime(value=datetime.datetime(2024, 6, 20, 9, 58, 16, 355237)), updated_at=D

# Create model

In [ ]:
from morpheus.project.types.discretization.spatial import Rotation
from morpheus.project.types.geometry import Polygon
from morpheus.project.types.Model import ModelId
from morpheus.project.application.write.Model import CreateModelCommand

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

n_cols = 100
n_rows = 50
rotation = 0.0

command = CreateModelCommand(
  project_id=project_id,
  user_id=user_id,
  model_id=ModelId.new(),
  geometry=polygon,
  n_cols=n_cols,
  n_rows=n_rows,
  rotation=Rotation.from_float(rotation),
)

project_command_bus.dispatch(command)

# Read Latest Model

In [ ]:
from morpheus.project.application.read.ModelReader import ModelReader

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)
model

## Update Model Geometry


In [ ]:
from morpheus.project.application.write.Model import UpdateModelGeometryCommand

new_polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922, 50.965),
    (13.925, 50.965),
    (13.925, 50.966),
    (13.922, 50.966),
    (13.922, 50.965)
  ]]
)

command = UpdateModelGeometryCommand(
  project_id=project_id,
  geometry=new_polygon,
  user_id=user_id
)

project_command_bus.dispatch(command)

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)
model.spatial_discretization.geometry

## Update Model Grid Properties

In [ ]:
from morpheus.project.application.write.Model import UpdateModelGridCommand

new_del_col_relative = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
new_del_row_relative = [0, 0.2, 0.4, 0.6, 0.8, 1]

command = UpdateModelGridCommand(
  project_id=project_id,
  user_id=user_id,
  n_cols=len(new_del_col_relative),
  n_rows=len(new_del_row_relative),
  rotation=10,
)

project_command_bus.dispatch(command)

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id)
print(model.spatial_discretization.geometry)
print(model.spatial_discretization.grid)
print(model.spatial_discretization.affected_cells.to_dict())

## Update Model Affected Cells

In [ ]:
from morpheus.project.application.write.Model import UpdateModelAffectedCellsCommand

model_reader = ModelReader()
model = model_reader.get_latest_model(project_id=project_id)
affected_cells = model.spatial_discretization.affected_cells

assert affected_cells.is_active(col=0, row=0) == False
affected_cells.set_active(col=0, row=0)
assert affected_cells.is_active(col=0, row=0) == True

command = UpdateModelAffectedCellsCommand(
  project_id=project_id,
  affected_cells=affected_cells,
  user_id=user_id
)

project_command_bus.dispatch(command)

In [ ]:
model_reader = ModelReader()
model = model_reader.get_latest_model(project_id=project_id)
affected_cells = model.spatial_discretization.affected_cells
assert affected_cells.is_active(col=0, row=0) == True

## Show Versions of Model so far

In [ ]:
model_reader = ModelReader()
model_reader.get_versions(project_id=project_id)

## Tag Version

In [ ]:
from morpheus.project.application.write.Model import CreateModelVersionCommand
from morpheus.project.types.ModelVersion import VersionTag, VersionDescription

command = CreateModelVersionCommand(
  project_id=project_id,
  version_tag=VersionTag('v1.0.0'),
  version_description=VersionDescription('Version 1.0.0'),
  user_id=user_id
)

project_command_bus.dispatch(command)

In [ ]:
model_reader.get_versions(project_id=project_id)

In [ ]:
version = model_reader.get_version_by_tag(project_id=project_id, tag=VersionTag('v1.0.0'))
version

In [ ]:
from morpheus.project.application.write.Model import UpdateModelVersionDescriptionCommand

command = UpdateModelVersionDescriptionCommand(
  project_id=project_id,
  version_id=version.version_id,
  version_description=VersionDescription('Version 1.0.0 - Updated'),
  user_id=user_id
)

project_command_bus.dispatch(command)

In [ ]:
version = model_reader.get_version_by_tag(project_id=project_id, tag=VersionTag('v1.0.0'))
version.description.to_str()

# Add well boundary

In [ ]:
from morpheus.project.application.write.Model.AddModelBoundary import AddModelBoundaryCommandPayload
from morpheus.project.application.write.Model import AddModelBoundaryCommand

payload: AddModelBoundaryCommandPayload = {
  'project_id': project_id.to_str(),
  'model_id': model.model_id.to_str(),
  'boundary_type': 'well',
  'boundary_geometry': {
    'type': 'Point',
    'coordinates': [13.923, 50.965]
  },
}

project_command_bus.dispatch(AddModelBoundaryCommand.from_payload(user_id=user_id, payload=payload))

# Read Model Boundaries

In [ ]:
model = model_reader.get_latest_model(project_id=project_id)
model.boundaries

# Remove Well Boundary

In [ ]:
from morpheus.project.application.write.Model.RemoveModelBoundary import RemoveModelBoundaryCommandPayload, RemoveModelBoundaryCommand

boundary_id = model.boundaries[0].boundary_id

payload: RemoveModelBoundaryCommandPayload = {
  'project_id': project_id.to_str(),
  'model_id': model.model_id.to_str(),
  'boundary_id': boundary_id.to_str(),
}

project_command_bus.dispatch(RemoveModelBoundaryCommand.from_payload(user_id=user_id, payload=payload))

In [ ]:
model = model_reader.get_latest_model(project_id=project_id)
model.boundaries